[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/keisen/tf-keras-vis/blob/master/examples/visualize_dense_layer.ipynb)

# Visualizing Dense layer using ActivationMaximization

## Preparation

### Install libraries

At first, when you didn't yet installed `tf-keras-vis` in your environment such Google Colab, please run the cell below.

In [ ]:
!pip install --upgrade pip
!pip install --upgrade tf-keras-vis tensorflow matplotlib

### Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
from tf_keras_vis.utils import num_of_gpus

_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

### Load tf.keras.Model

This tutorial use VGG16 model in tf.keras but if you want to use other tf.keras.Models, you can do so by modifying section below.

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16 as Model

# Load model
model = Model(weights='imagenet', include_top=True)
model.summary()

## Visualizing a specific output category

### Define a function to modify the model

First, when the softmax activation function is applied to the last layer of model, it may obstruct ActivationMaximization, so you need to replace the function to a linear function. Here, we will do so using `model_modifier`.

In [ ]:
def model_modifier(cloned_model):
    cloned_model.layers[-1].activation = tf.keras.activations.linear
    return cloned_model

### Create ActivationMaximization Instnace

When `clone` argument is True(default), the `model` will be cloned, so the `model` instance will be NOT modified, but it may take a while.
Here, because it does NOT need to do so, setting False.

In [ ]:
from tf_keras_vis.activation_maximization import ActivationMaximization

activation_maximization = ActivationMaximization(model,
                                                 model_modifier,
                                                 clone=False)

### Define Score function

You MUST define `score` function that return arbitrary category value. Here, we try to visualize a category as defined No.20 (ouzel) of imagenet.

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore

score = CategoricalScore(20)

# Instead of CategoricalScore function object, you can define the scratch function such as below:
def score_function(output):
    # The `output` variable refer to the output of the model,
    # so, in this case, `output` shape is `(3, 1000)` i.e., (samples, classes).
    return output[:, 20]

### Visualize

ActivationMaximization will maximize the model output value that is computed by the score function. Here, we try to visualize a convolutional filter.

In [ ]:
%%time
from tf_keras_vis.activation_maximization.callbacks import PrintLogger

activations = activation_maximization(score,
                                      callbacks=[PrintLogger(interval=50)])

subplot_args = { 'nrows': 1, 'ncols': 1, 'figsize': (4, 4),
                 'subplot_kw': {'xticks': [], 'yticks': []} }
f, ax = plt.subplots(**subplot_args)
ax.imshow(activations[0])
ax.set_title('Ouzel', fontsize=14)
plt.tight_layout()
plt.show()

## Visualizing specific output categories

Then, let's visualize multiple categories at once!

### Define Loss function

You MUST define loss function that return arbitrary category values. Here, we try to visualize categories as defined No.1 (Goldfish), No.294 (Bear) and No.413 (Assault rifle) of imagenet.
Here, we use `SmoothedCategoricalScore` that add noise to score to smooth score funtion and then accomplish getting clear image.

In [ ]:
from tf_keras_vis.utils.scores import SmoothedCategoricalScore

image_titles = ['Goldfish', 'Bear', 'Assault rifle']
scores = SmoothedCategoricalScore([1, 294, 413])

### Create SeedInput values

And then, beforehand, you MUST create seed-input value. In default, when visualizing a specific output category, tf-keras-vis automatically generate seed-input to visualize a image. When visualizing multiple output categories, you MUST manually create seed-inputs whose samples-dim is as many as the number of the filters you want to generate.

In [ ]:
# Create seed inputs whose shape is (samples, height, width, channels).

seed_input = tf.random.uniform((3, 224, 224, 3), -1., 1.)

### Visualize

Here, we will visualize 3 images while `steps` option is to be 512 to get clear images.

In [ ]:
%%time

# Generate max activation
activations = activation_maximization(scores,
                                      seed_input=seed_input,
                                      steps=512,
                                      callbacks=[ PrintLogger(interval=50)])

# Render
subplot_args = { 'nrows': 1, 'ncols': 3, 'figsize': (12, 4),
                 'subplot_kw': {'xticks': [], 'yticks': []} }
f, ax = plt.subplots(**subplot_args)
for i, title in enumerate(image_titles):
    ax[i].set_title(title, fontsize=14)
    ax[i].imshow(activations[i])
plt.tight_layout()
plt.savefig('images/visualize-dense-layer')
plt.show()